In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import joblib

In [18]:
import tensorflow as tf

#não estou conseguindo fazer funcionar, mas nem é necessário já q são dados tabulares e são poucos.
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        #não sei pra q serve mas tá escrito q isso faz com que o TensorFlow pegue memória aos poucos, e não toda de uma vez.
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        print(f"✅ GPU Encontrada: {gpus}")
        print("O treinamento será acelerado pela placa de vídeo!")
        
    except RuntimeError as e:
        print(e)
else:
    print("❌ Nenhuma GPU detectada. O treinamento será feito na CPU (mais lento).")

❌ Nenhuma GPU detectada. O treinamento será feito na CPU (mais lento).


In [19]:
data = pd.read_csv('paddydataset.csv')

In [20]:
#separar Features (X) e alvo (y)
#por enquanto o alvo é a última coluna: 'Paddy yield(in Kg), talvez mudar?'
X = data.drop('Paddy yield(in Kg)', axis=1)
y = data['Paddy yield(in Kg)']

In [21]:
#pré-processamento Identificar colunas numéricas e categóricas, tava escrito pra fazer só fiz
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

In [22]:
#autobots rollout (parte dos transformers já q a gente tem mt variável categórica
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ])

X_processed = preprocessor.fit_transform(X)

In [23]:
#clásico sem mt firula dividindo em Treino e Teste
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [24]:
#Rede Neural bem basiquinha sem mt presepada
model = Sequential()
#Camada de entrada (o input_dim depende das colunas processadas)
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
#Camadas ocultas
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
#Camada de saída (1 neurônio pois é regressão - queremos 1 valor de yield)
model.add(Dense(1, activation='linear'))

#Compilar o modelo
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Treinar
print("Treinando o modelo...")
# CORREÇÃO: Removido o .toarray()
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Avaliar
# CORREÇÃO: Removido o .toarray() aqui também
loss, mae = model.evaluate(X_test, y_test)
print(f"Erro Médio Absoluto (MAE) no teste: {mae:.2f} Kg")

#salvar o modelo e o pre-processador
model.save('modelo_paddy.h5')
joblib.dump(preprocessor, 'preprocessor.pkl')
print("Modelo e pre-processador salvos!")

Treinando o modelo...
Epoch 1/50


C:\Users\renat\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


56/56 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 593471168.0000 - mae: 22555.6641 - val_loss: 576837184.0000 - val_mae: 22103.8672
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 591121024.0000 - mae: 22510.9375 - val_loss: 570805376.0000 - val_mae: 21988.8965
Epoch 3/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 573729152.0000 - mae: 22186.7852 - val_loss: 537845312.0000 - val_mae: 21369.5176
Epoch 4/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 509742720.0000 - mae: 20967.1699 - val_loss: 441660384.0000 - val_mae: 19473.4766
Epoch 5/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 367540928.0000 - mae: 17940.9863 - val_loss: 267676576.0000 - val_mae: 15412.7080
Epoch 6/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 174433792.0000 - mae: 12515.3916 - val_loss: 93052304.0000 - val_mae: 9301.1533
Epoch 7/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 47265240.0000 - mae: 6349.5913 - val_loss: 23433832.0000 - val_mae: 4362.4072
Epoch 8/50
56/56 ━━━━━━━━━━━━━━━━━━━━

Erro Médio Absoluto (MAE) no teste: 730.39 Kg
Modelo e pre-processador salvos!
